# Differential Expression Analysis - RhithroLoxo

## Set-up

If you've already saved the workspace image from a previous session, jupyter should automatically reload it. You may need to reload the packages though. The .RData file is not on GitHub, so you will have to actually run it the first time through. 

First, make sure you're actually running this from a compute node, not the login. On Poseidon, logins are 'l1' and 'l2', whereas all other nodes start with 'pn'.

In [ ]:
Sys.info()

Clear environment (if you want to get rid of objects from previous sessions). Or load .RData if you want to recover workspace. (Just note that some objects get overwritten throughout script, so will reflect only the one written last.)

In [ ]:
rm(list = ls())
#load(".RData")

Now load in the packages. Specify number of cores you requested for SLURM interactive session under `register(MulticoreParam())`

In [ ]:
require(DESeq2)
require(ggplot2)
require(apeglm)
require(ashr)
library("BiocParallel")
register(MulticoreParam(34))
require(VennDiagram)
require(RColorBrewer)
require(pheatmap)
require(dplyr)
require(tibble)
require(tidyr)

## Data import

Now import the count data, rounding decimals to integers.

In [ ]:
path_to_main <- "/vortexfs1/scratch/ztobias/RhithroLoxo_DE/" #change accordingly based on parent file structure
path_to_counts <- "outputs/quant/salmon.isoform.counts.matrix"
path <- paste(path_to_main,path_to_counts,sep="")
all_counts <- read.table(path,header=TRUE)
all_counts <- round(all_counts)
all_counts[] <- sapply(all_counts, as.integer)

Take a look.

In [ ]:
head(all_counts)
dim(all_counts)

In [ ]:
all_counts["TRINITY_DN34991_c0_g1_i1",]

Read in the sample metadata and have a look.

In [ ]:
path_to_meta <- paste(path_to_main,"metadata/DESeq2_coldata.txt",sep="")
coldata <- read.table(path_to_meta,header=FALSE,row.names=1)
colnames(coldata) <- c("site","condition","range","sex")
head(coldata)
dim(coldata)

Make sure the two matrices contain all of the same samples and are in order.

In [ ]:
all(rownames(coldata) == colnames(all_counts))

## Filtering

EXPLAIN HERE

Create a DESeq dataset just for the purpose of calculating normalized counts

In [ ]:
filtering <- DESeqDataSetFromMatrix(countData = all_counts, colData = coldata, design = ~ condition)

Estimate size factors and save normalized counts to object 'norm_mat'

In [ ]:
filtering <- estimateSizeFactors(filtering)
norm_mat <- counts(filtering, normalized=TRUE)

Then let's remove the parasitized crabs. This includes all samples with the naming pattern `*_P_*`. The function `grepl()` returns a boolean vector that can be used to index. I will also remove sample MD_C_12, as this was identified to have a latent infection (unidentified infection detected in previous runs of this analysis).

In [ ]:
norm_mat_sub <- norm_mat[,!grepl("*_P_*", colnames(norm_mat))]
norm_mat_sub <- subset(norm_mat_sub, select=-c(MD_C_12))

Verify the names you wanted to remove are gone.

In [ ]:
colnames(norm_mat_sub)

Column numbers of the parasite samples in `norm_mat_sub` are 19 and 28. Command below finds the index of the maximum column for each row, checks if it matches 19 or 28 (the parasite samples), returns boolean which is used to subset the dataframe.  Let's take a look at a slice of the output to verify it's behavior. Parasite samples follow the naming pattern `*_F_*`.

In [ ]:
contam_subset <- norm_mat_sub[max.col(norm_mat_sub) %in% c(19,28),]
dim(contam_subset)
contam_subset[1:20,18:29]

Scrolling through, it's clear that these 5757 transcripts have the highest expression in at least one of the parasite samples. This is indicative of Loxo contamination in the Rhithro assembly. However, because the Loxo externae are female, it could be that by pulling out these transcripts, I am removing transcripts that are generally overexpressed in females relative to males in crustaceans more broadly (that cross-map between Loxo and Rhithro). This could interfere with the investigation of sex-specific differences of infection later on, i.e. if we see a greater/different transcriptional response in males, is it due to host feminization or because we have pulled out these female-specific transcripts early in the pipeline? To investigate this possibility, I'll make a DESeqDataSet using just these transcripts and compare uninfected males and females. 

First I'll remove the actual Loxo externae samples from the original count data.  From the sample metadata as well. Then I'll subset the count data to include just the putative Loxo contamination.

In [ ]:
loxo_counts <- all_counts[rownames(contam_subset),]
loxo_counts <- loxo_counts[,-c(29,41)]
coldata <- coldata[-c(29,41),]

We'll also reset the factor levels for `coldata` since we've removed the parasite samples. We'll keep female as the reference level. While we're at it, let's set the factor levels for range to be Native, Invasive, Absent. 

In [ ]:
coldata$sex <- factor(coldata$sex, c("F","M"))
coldata$range <- factor(coldata$range, c("Native","Invasive","Absent"))
coldata$sex
coldata$range

Cool. Now we can actually go on to seeing if there is any bias by sex when looking at just the putative Loxo contaminant transcripts. 

In [ ]:
dds_loxo <- DESeqDataSetFromMatrix(countData = loxo_counts, colData = coldata, design = ~ condition )
dds_loxo$group <- factor(paste0(dds_loxo$sex, dds_loxo$condition))
design(dds_loxo) <- ~ group
dds_loxo <- DESeq(dds_loxo, parallel=TRUE)

In [ ]:
resultsNames(dds_loxo)

In [ ]:
dds_loxo_res <- results(dds_loxo, contrast=c("group", "MC", "FC"), alpha=0.05)
summary(dds_loxo_res)

Okay so there are 17 transcripts among those that are putative Loxo contaminants that are differentially expressed between uninfected males and females. Let's take a look at some of these.

In [ ]:
data.frame(subset(dds_loxo_res, padj < 0.05))

I'll just use the code below to plot select transcripts. Change gene name as desired in the `plotCounts` function. (This basic ggplot structure will be used throughout and written into custom functions later.)

In [ ]:
count_plot <- plotCounts(dds_loxo, gene="TRINITY_DN30863_c0_g1_i1", intgroup=c("condition", "site", "sex"), returnData=TRUE)
ggplot(count_plot, aes(x=sex, y=count, color=site)) +
    facet_grid(.~condition) +
    theme(strip.text.x = element_text(size = 12), 
    axis.text.x = element_text(size=11), 
    axis.text.y = element_text(size=11)) +
    xlab("") +
    ylab("") +
    geom_point(position=position_jitter(w=0.2,h=0), show.legend = TRUE) + 
    geom_smooth(method = "lm", se=F, aes(group=1), show.legend = FALSE, color = "blue") +
    stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="pointrange", color="red") +
    stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="errorbar", color="red", width=0.2) +
    scale_y_log10()

While some over these are underexpressed in males relative to females, which would indicate that they might be reflective of general sex-specific expression among crustaceans, the number is pretty insignificant relative to the patterns that will emerge later on regarding sex-specific response to infection. I am going to proceed with discarding these putative Loxo contaminant transcripts. Furthermore, many of these transcripts have very low mean expression, and will be discarded later on during an expression-level filtering anyway. 

Okay now I'm going to save the rownames of the putative Loxo contamination for use later:

In [ ]:
contam_IDs <- rownames(contam_subset)
length(contam_IDs)

Here are the 5757 transcripts that have higher expression in a parasite sample than any of the "clean" samples. I am going to move on to a base parasitized vs. control analysis for the purpose of further filtering, for reasons that will become apparent. I'll include site in the design to account for site-specific differences. First we have to remove the loxo samples from `all_counts`, as we did for the loxo contaminant subset.

In [ ]:
all_counts <- all_counts[,-c(29,41)]
dds <- DESeqDataSetFromMatrix(countData = all_counts, colData = coldata, design = ~ site + condition)
dds <- DESeq(dds, parallel=TRUE)

Let's visualize the data using some PCAs. This will be helpful for identifying sample outliers.

First we'll apply a variance stabilizing transformation to our normalized counts. 

In [ ]:
vsd <- vst(dds, blind=FALSE)

In [ ]:
PCA <- plotPCA(vsd, intgroup="condition")
PCA + geom_label(aes(label = name))

You can see that PCA 1 clearly separates the sample according to infection status (whether this is due to a crab response or contamination from parasite remains to be seen...). Along the second PCA axis, you can see four extreme outlier at the upper left corner. Let's investigate this a little more closely by looking at the expression of particular high variance transcripts. 

In [ ]:
topVarGenes <- head(order(rowVars(assay(vsd)), decreasing = TRUE), 500)
df <- as.data.frame(colData(dds)[,c("condition","site")])
vsd_df <- assay(vsd)
heatmap <- pheatmap(vsd_df[topVarGenes,], cluster_rows=TRUE, show_rownames=FALSE,
         cluster_cols=TRUE, annotation_col=df)
heatmap

Looking at the top 500 highest variance transcripts, you can see that the transcript cluster in the mid-top left separates samples in the same way as PCA axis 2 from above, with MA_C_1, MA_C_2, MA_C_4, and AP_C_6 having really high expression. Let's figure out what those are. 

In [ ]:
idx <- sort(cutree(heatmap$tree_row, k=10)) #separate transcripts by cluster assignment
idx <- names(which(idx==7)) #after searching, cluster 7 is the one with the transcripts we want
heatmap <- pheatmap(vsd_df[idx,], cluster_rows=TRUE, show_rownames=FALSE, cluster_cols=TRUE)
heatmap

Okay now we found the cluster that has all of these transcripts. Let's get the names and search annotations to see if there is anything we can tell about these transcripts. Load in annotations.

In [ ]:
annot <- read.table("../EnTAP/entap_outfiles/similarity_search/DIAMOND/overall_results/best_hits_lvl0.tsv", sep="\t", fill=TRUE, header=TRUE, quote="", stringsAsFactors = FALSE)

In [ ]:
outlier_annot <- annot[annot[,1] %in% rownames(vsd[idx,]),] #get matching annotations
head(outlier_annot[order(outlier_annot[,11]),c(1,2,3,11,12,13,14)], 20) #show results

There doesn't seem to be anything special about these transcripts, though it's hard to tell just from the reference IDs. 

I am going to remove these outlier samples. They are contributing too much variation and will present issues when trying to fit the DESeq2 models and cause issues with count normalization. 

In [ ]:
all_counts <- all_counts[,!(colnames(all_counts) %in% c("MA_C_1","MA_C_2","MA_C_4","AP_C_6"))]
all_counts[1:6,1:6]
dim(all_counts)

Okay now you can see the four of them were removed. Now make sure the coldata matches.

In [ ]:
coldata <- coldata[colnames(all_counts),]
all(rownames(coldata) == colnames(all_counts))

Okay good let's make another `dds` object to continue with our filtering. We'll just overwrite the existing one.

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = all_counts, colData = coldata, design = ~ site + condition)

In [ ]:
dds <- DESeq(dds, parallel=TRUE)

In [ ]:
res <- results(dds, alpha=0.05)
summary(res)

Here we see that there are 1556 transcripts deemed significantly upregulated in parasitized crabs, and 1452 significantly downregulated, according the the Wald test.

As mentioned earlier, it is my suspicion that some of these upregulated transcripts are contamination from Loxo. Let's have a look.

In [ ]:
plotMA(res, ylim=c(-30,15))

Here we see the pattern I have alluded to, where there is a cloud of extremely overexpressed (>10 LFC) in the infected relative to the control. It is my suspicion that these represent contamination from Loxo rather than an actually response by the crabs. 

We also see a cloud of extremely *under*expressed transcripts (<-20 LFC). I will deal with these after the Loxo contamination (but I can tell you now that these are contamination from trematodes and perhaps other parasites, mostly in three NH control crabs \[hence the lower count means\]). They are underexpressed in parasitized crabs (overexpressed in control) because the transcriptome was made from just control crabs. 

Let's see if the *over*expressed ones match the transcript IDs we pulled out earlier as possible Loxo contaminant transcripts. 

First let's make an ordered data.frame of the significant transcripts.

In [ ]:
res_sig <- data.frame(subset(res, padj < 0.05))
res_sig <- res_sig[order(-res_sig$log2FoldChange),]
head(res_sig)

You can see that there are some extremely significant, extremely overexpressed transcripts at the top, many of which have high rates of expression overall. These are likely contamination from Loxo. I am not sure how they made it into the transcriptome. It's hard to think of a biological reason. I am thinking this is due to index hopping during sequencing. If there were parasitizied and unparasitized crabs on the same lane (there were), then even a small amount of index hopping could have led to parasite reads being built into the txm, either as chimeras with crab sequences or alone. 

I am going to compare the list of transcripts I made earlier to these to see how much of an overlap there is. 

There are 3008 significant transcripts, 1556 up and 1452 down. There were 5757 transcripts that had higher expression in one of the two parasite samples than any of the clean samples. Let's look at the intersect. 

In [ ]:
contam_intersect <- intersect(contam_IDs, rownames(res_sig))
length(contam_intersect)

Okay so there are 472 transcripts that came up as significant that were also identified as potential contamination. Let's see what the distribution of LFCs look like for these transcripts.

In [ ]:
hist(res_sig[(rownames(res_sig) %in% contam_intersect),2])

It shows a bimodal pattern. Most of these transcripts are overexpressed, but a subset (~100) are extremely enriched. It is these that are most likely to be contamination. We will still consider all of these putative contamination and slate them for removal, as I believe it is more prudent to be liberal with what we discard.

 Let's take a look at the significant results table with the putative Loxo contaminants removed.

In [ ]:
head(res_sig[!(rownames(res_sig) %in% contam_intersect),],n=10)

A lot of those transcripts are now removed. There are just around 7 of them left, depending on where you draw the line.

I am going to repeat visualization with all of putative Loxo contaminant transcripts removed. 

In [ ]:
res2 <- results(subset(dds, !rownames(dds) %in% contam_IDs), alpha=0.05)

In [ ]:
plotMA(res2, ylim=c(-30,15))

You can see now that the big cloud of points at the top right has mostly disappeared. A few points remain. I am still going to consider these as contamination and remove them manually by thresholding.

But first, let's look at the identity of the *under*expressed transcripts. Let's make a subset of the results without these transcripts, and then get the names of those transcripts. 

In [ ]:
extra_under <- subset(res2, log2FoldChange < -15)
extra_under <- rownames(extra_under)
length(extra_under)

Okay so there are 400 that are extremely underexpressed.

Now we'll make another vsd object and use the list of transcripts to plot just those that had extremely low LFCs.

In [ ]:
df <- as.data.frame(colData(dds)[,c("condition","site")])
vsd <- vst(dds, blind=TRUE)
vsd_df <- assay(vsd)
heatmap3 <- pheatmap(vsd_df[extra_under,], cluster_rows=TRUE, show_rownames=FALSE,
         cluster_cols=TRUE, annotation_col=df)
heatmap3

It's clear that expression of these transcripts occurs in just a few crabs. It's difficult to see the sample names, but it is mostly in three control samples from NH (NH_C_8, NH_C_11, NH_C_13) and one from NJ (NJ_C_10). So these transcripts are only represented in a few samples. 

Let's use the annotations to figure out the identity of these transcripts.

In [ ]:
low_annot <- annot[annot$Query.Sequence %in% extra_under,c(1,2,3,11,13,14)]
low_annot <- low_annot[order(low_annot$E.Value),]
low_annot

The taxonomic identity of the matches is not revealing here. However, on previous runs of the annotation pipeline, I did not remove the sequences listed as contamination by EnTAP from the transcriptome prior to creating the expression matrix. When these were left in, the vast majority of the transcripts in this underexpressed cloud were flatworm, nematode, and fungal sequences, all known pathogens of crabs. I decided to remove these contaminants earlier in the pipeline, so their identity is not revealed here. If you aren't convinced, skip the `remove_EnTAP_contam` step in the Snakemake pipeline. 

Okay, now I am going to remove all of these transcripts, not just those that map because many are lacking annotation. For removing both the putative Loxo (upper) contaminants, I am going to get the rownames of the 7 remaining transcripts that fell into that cloud and exclude them. For the other pathogen (lower) contaminants, I am going to set a threshold of -15 to remove the whole cloud. Let's look and see what that would remove, after already having flagged the putative Loxo for removal.

In [ ]:
plotMA(res2, ylim=c(-30,15))
abline(h=9,col="red")
abline(v=100,col="red")
abline(v=-20,col="red")

So I'm going to get rid of the 7 transcripts in the very upper right section. These are the putative Loxo contaminants that fell into that cloud of points that had mostly been previously identified as Loxo contamination from the expression comparison. And I am going to get rid of everything below the -15 line. These are the other pathogen contaminants. I got the transcript names for the 7 transcripts in the upper right from the first 7 entries of the dataframe displayed after the histogram above (which shows the extremely overexpressed transcripts that didn't intersect with the loxo contam list, but nonetheless are likely contamination).

In [ ]:
contam_high <- rownames(head(res_sig[!(rownames(res_sig) %in% contam_intersect),],n=7))
contam_low <- rownames(subset(res2, res2$log2FoldChange < -15))
contam_add <- c(contam_high,contam_low)
length(contam_add)

Okay here are the 407 remaining transcripts we deem to be contamination. I am going to add them to the list of contam_IDs, which will be used for removal from the intial counts object. Then the analysis will be re-run.

This is all to account for renormalization after removal, since many of these contigs had high mean expression across samples, especially the Loxo contaminants. I also want to have all of the putative contaminant transcripts removed before I do the WGCNA analysis. Because it looks for co-expression patterns among transcripts, if I leave in transcripts that are actually just contaminants, it will likely assign transcripts to modules not based on any functional relevance to particular pathways, but rather just to infection status. 

In [ ]:
contam_IDs <- c(contam_IDs, contam_add)
length(contam_IDs)

Okay so there are 6164 transcripts that we are going to remove because they were flagged as contamination. Let's do that now.

In [ ]:
counts_clean <- all_counts[!rownames(all_counts) %in% contam_IDs,]
dim(counts_clean)

So we've removed 6164 from the original 146332, for 140168. Now I am going to do expression-level based filtering to removed lowly expressed transcripts. I am doing this for a few reasons. First, such lowly expressed transcripts are ostensibly less biologically meaningful. Second, they are more likely to be latent contamination from other taxa. Third, they were less likely to be annotated and thus are uninformative. And lastly, on previous runs of this analysis, it was apparent that some transcripts that were deemed significantly differentially expressed were due to extremely high expression in just a small handful of samples, with zero or very minimal expression in other samples. While such transcripts were already revealed above, this comes into play later when looking at the effect of infection *within* ranges. In this instance, contrasting patterns among ranges precluded them from being removed in the procedure above. Furthermore, WGCNA will use a pared down expression matrix, as [suggested](https://horvath.genetics.ucla.edu/html/CoexpressionNetwork/Rpackages/WGCNA/faq.html) by the creators. Removing them now will allow me to conduct DE analysis and WGCNA on the identical dataset. 

I am going to remove all transcripts that have less than a count of 10 in over 90% of samples (69). Again, I'll make a DESeqDataSet object for the purpose of normalizing counts.

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = counts_clean, colData = coldata, design = ~ condition)

Then I'll get the normalized counts and save it to the `norm_mat` object, overwriting.

In [ ]:
dds <- estimateSizeFactors(dds)
norm_mat <- counts(dds, normalized=TRUE)

Here I'll get the dimensions before and after removal to see how many were removed.

In [ ]:
dim(norm_mat)
remove <- rowSums(norm_mat < 10 ) > 69
norm_mat <- norm_mat[!remove,]
dim(norm_mat)

So this would result in 60488 transcripts. Still a lot. You can see that a huge number of transcripts (79680 to be precise) were only meaningfully expressed in a small number of samples. Their removal should help in cutting through any confouding signals generated from processes/contamination in just a handful of crabs.

Now we'll remove it from the main count matrix and finally proceed with the actual analyses!

In [ ]:
counts_clean <- counts_clean[rownames(norm_mat),]
dim(counts_clean)

## Parasitized vs. control differential expression 

Here we'll just do a base comparison between infected and uninfected, using all 77 remaining samples. Let's just verify that everything matches up.

In [ ]:
all(rownames(coldata) == colnames(counts_clean))

Good. Create new dds object without the contaminant and lowly expressed transcripts.

In [ ]:
dds_clean <- DESeqDataSetFromMatrix(countData = counts_clean, colData = coldata, design = ~site + condition)

Here we'll use a likelihood ratio test to control for site-specific effects.

In [ ]:
dds_clean <- DESeq(dds_clean, parallel=TRUE, test="LRT", reduced=~site)

In [ ]:
resultsNames(dds_clean)

In [ ]:
res_clean <- results(dds_clean, alpha=0.05) #by default, it takes the last factor (P vs. C)
summary(res_clean)

Now you can see that we have 852 significantly upregulated transcripts and 743 significantly downregulated transcripts.

Plot the LFCs

In [ ]:
plotMA(res_clean, ylim=c(-12,12))

Save the significant results:

In [ ]:
res_clean_sig <- data.frame(subset(res_clean, padj < 0.05))
res_clean_sig <- res_clean_sig[order(res_clean_sig$padj),]
head(res_clean_sig,20)

Now I just want to count how many significant transcripts had LFCs above 2. 

In [ ]:
nrow(res_clean_sig[res_clean_sig$log2FoldChange>(2),])
nrow(res_clean_sig[res_clean_sig$log2FoldChange<(-2),])
nrow(res_clean_sig[abs(res_clean_sig$log2FoldChange)>(2),])

In [ ]:
head(res_clean_sig)

Okay 72 significantly above 2 LFC and 62 significantly below -2 LFC. 

Now I'm going to make another PCA, now that the potential contaminants have been removed.

In [ ]:
vsd <- vst(dds_clean, blind=FALSE)
PCA <- plotPCA(vsd, intgroup="condition")
PCA + geom_label(aes(label = name))

You can see now that they don't separate out. This is expected, as there is a lot of other variation present in the count data. This does not mean there aren't differences, however, as was shown above in the results summary. 

Let's look at the expression patterns of select transcripts with decapod annotations. 

In [ ]:
decapod_annot <- annot[grepl("decapoda",annot[,15]),]

In [ ]:
head(res_clean_sig[rownames(res_clean_sig) %in% (decapod_annot[,1]),], 30)
dim(res_clean_sig[(rownames(res_clean_sig) %in% decapod_annot[,1]),])

Below I am going to export a selection of counts plots of most significant transcripts, using only those with informative decapod annotations (no hypothetical or unknown proteins). I wrote some simple functions to plot the counts, add annotations manually, and then export a panel type figure using cowplot.

In [ ]:
library(cowplot)
easy_plotCounts <- function(ID){
    count_plot <- plotCounts(dds_clean, gene=ID, intgroup=c("condition", "site"), returnData=TRUE)
    ggplot(count_plot, aes(x=condition, y=count, color=site)) +
        theme(strip.text.x = element_text(size = 12), 
        axis.text.x = element_text(size=11), 
        axis.text.y = element_text(size=11)) +
        xlab("") +
        ylab("") +
        geom_point(position=position_jitter(w=0.2,h=0), show.legend = FALSE) + 
        geom_smooth(method = "lm", se=F, aes(group=1), show.legend = FALSE, color = "blue") +
        stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="pointrange", color="red") +
        stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="errorbar", color="red", width=0.2) +
        scale_y_log10()
}

add_title <- function(ID, title){
     easy_plotCounts(ID) + 
        ggtitle(title) + 
        theme(plot.title = element_text(hjust=0.5,size=14,face="bold"), legend.position="none")
}

In [ ]:
rownames(decapod_annot) <- decapod_annot[,1]

In [ ]:
options(repr.matrix.max.rows=269, repr.matrix.max.cols=10) #change display options to show entire table
na.omit(decapod_annot[rownames(res_clean_sig),c(3,11,13,14)])
options(repr.matrix.max.rows=50, repr.matrix.max.cols=10)

In [ ]:
p1 <- add_title("TRINITY_DN5780_c0_g1_i1","laminin subunit alpha-4-like")
p2 <- add_title("TRINITY_DN4017_c2_g1_i1","cellular retinoic acid-binding protein 1-like")
p3 <- add_title("TRINITY_DN4300_c0_g2_i2","glycogen debranching enzyme")
p4 <- add_title("TRINITY_DN93978_c0_g1_i1","V-type proton ATPase proteolipid subunit")
p5 <- add_title("TRINITY_DN20560_c0_g1_i1","C-type lectin 1")
p6 <- add_title("TRINITY_DN2676_c0_g1_i1","ficolin-2-like")
p7 <- add_title("TRINITY_DN4911_c0_g1_i1","hemolymph clottable protein")
p8 <- add_title("TRINITY_DN440_c0_g1_i1","D-3-phosphoglycerate dehydrogenase")
p9 <- add_title("TRINITY_DN34522_c1_g1_i1","dual oxidase maturation factor 1")
p10 <- add_title("TRINITY_DN96090_c0_g1_i1","dual oxidase 2-like")
p11 <- add_title("TRINITY_DN12148_c0_g1_i1","hemocytin-like")
p12 <- add_title("TRINITY_DN448_c0_g1_i6","PDGF/VEGF-related factor 1")


main <- plot_grid(p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12, nrow=4, ncol=3)
##legend <- get_legend(plot1  + guides(color = guide_legend(nrow = 1)) + theme(legend.position = "bottom"))
##done <- plot_grid(main, legend, nrow=4,ncol=3)

main

save_plot("../vis/DESeq2_top_PvC_genes.pdf", plot=main, nrow=4, ncol=3, base_asp=1.41)

Jupyter didn't render this plot well. Check the `vis/` directory for the output. 

Let's explore other counts.

In [ ]:
get <- "TRINITY_DN16709_c0_g1_i1"
annot[annot[,1]==get,c(1,2,3,11,12,13,14)]
easy_plotCounts(get)

## Sex-specific effects

Now we can do a similar test but use a different design in which we add in coefficients for sex and the interaction between sex and condition. We cannot include site in this comparison because there are sites for which there are not parasitized individuals of both sex. 

In [ ]:
dds_sex <- DESeqDataSetFromMatrix(countData = counts_clean, colData = coldata, design = ~condition + sex + sex:condition)

LRT against reduced model of just sex + condition.

In [ ]:
dds_sex <- DESeq(dds_sex, parallel=TRUE, test="LRT", reduced=~condition + sex)

Get results.

In [ ]:
res_sex <- results(dds_sex, alpha=0.05)
summary(res_sex)

Looks like there are 55 that show a significant interaction.

In [ ]:
res_sex_sig <- data.frame(subset(res_sex, padj < 0.05))
res_sex_sig <- res_sex_sig[order(res_sex_sig$padj),]
head(res_sex_sig, 20)

Find those with any annotation

In [ ]:
res_sex_sig[rownames(res_sex_sig)%in%annot[,1],]

Get select annotations.

In [ ]:
annot[annot[,1]=="TRINITY_DN8359_c0_g1_i1",c(1,2,3,11,12,13,14)]

Modify the plotting function from above.

In [ ]:
easy_plotCounts_sex <- function(ID){
    count_plot <- plotCounts(dds_sex, gene=ID, intgroup=c("condition", "site", "sex"), returnData=TRUE)
    ggplot(count_plot, aes(x=condition, y=count, color=site)) +
        facet_grid(.~sex) +
        theme(strip.text.x = element_text(size = 12), 
        axis.text.x = element_text(size=11), 
        axis.text.y = element_text(size=11)) +
        xlab("") +
        ylab("") +
        geom_point(position=position_jitter(w=0.2,h=0), show.legend = TRUE) + 
        geom_smooth(method = "lm", se=F, aes(group=1), show.legend = FALSE, color = "blue") +
        stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="pointrange", color="red") +
        stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="errorbar", color="red", width=0.2) +
        scale_y_log10()
}

In [ ]:
easy_plotCounts_sex("TRINITY_DN8359_c0_g1_i1")

Just checking to see the balance of sex/condition.

In [ ]:
nrow(coldata[coldata$sex=="F"&coldata$condition=="P",])
nrow(coldata[coldata$sex=="F"&coldata$condition=="C",])
nrow(coldata[coldata$sex=="M"&coldata$condition=="P",])
nrow(coldata[coldata$sex=="M"&coldata$condition=="C",])

As you can see, the sample sizes are uneven. We'll do the following to account for this when looking for differences between the sexes in the magnitude of their transcriptional response. 

Let's make Wald contrasts just looking at P vs C within each sex. However, because there are unequal sample sizes, we'll subsample to the smallest number of P and C crabs between each sex. We'll do this randomly over 1000 iteractions, in what is kind of like a rarefaction procedure. More like downsampling with randomization. This will take a really long time, so if you are re-running these analysis and would like to skip, just go directly to the next set of analyses.

First we'll add a column to our coldata to pair sex and condition into one term.

In [ ]:
coldata_raref <- coldata
coldata_raref$group <- factor(paste0(coldata_raref$sex, coldata_raref$condition))

Next is the loop that runs the analyses 1000 times. The end of this chunk saves the resulting objects, so just skip and load if you've already run. REMEMBER TO UNCOMMENT PRE-COMMIT

In [ ]:
#sex_raref <- data.frame(rep=integer(0),sex=character(0),up=integer(0),down=integer(0), stringsAsFactors = FALSE) #initialize
#DE_female <- list() #initialize list the will hold names of DE transcripts in each iteration
#DE_male <- list() #again
#for (i in 1:1000){
#    coldata_C <- coldata_raref[coldata_raref$condition == "C",] #subset to just control
#    coldata_C <- coldata_C %>% rownames_to_column('sample') %>% group_by(sex) %>% sample_n(size=21) %>% column_to_rownames('sample') #group by site and randomly sample 21 (lowest number of C samples in either sex (F))
#    coldata_P <- coldata_raref[coldata_raref$condition == "P",] #subset to just parasitized
#    coldata_P <- coldata_P %>% rownames_to_column('sample') %>% group_by(sex) %>% sample_n(size=14) %>% column_to_rownames('sample') #group by site and randomly sample 2 (lowest number of P samples in either sex (M))
#    coldata_iter <- rbind(coldata_C,coldata_P) #combine dfs
#    counts_clean_raref <- counts_clean[,match(rownames(coldata_iter), colnames(counts_clean))] #subset count data to match included samples
#    dds_sex_raref <- DESeqDataSetFromMatrix(countData = counts_clean_raref, colData = coldata_iter, design = ~group)
#    dds_sex_raref <- DESeq(dds_sex_raref, parallel=TRUE, quiet=TRUE)
#    res_FPvFC <- results(dds_sex_raref, contrast=c("group", "FP", "FC"), alpha=0.05)
#    res_MPvMC <- results(dds_sex_raref, contrast=c("group", "MP", "MC"), alpha=0.05)
#    sex_raref <- rbind(sex_raref, list(rep=i,sex="F",up=nrow(res_FPvFC[which(res_FPvFC$log2FoldChange>0 & res_FPvFC$padj<0.05),]),down=nrow(res_FPvFC[which(res_FPvFC$log2FoldChange<0 & res_FPvFC$padj<0.05),])), stringsAsFactors=FALSE) #append rep, sex, and # up and down sig DE transcripts
#    sex_raref <- rbind(sex_raref, list(rep=i,sex="M",up=nrow(res_MPvMC[which(res_MPvMC$log2FoldChange>0 & res_MPvMC$padj<0.05),]),down=nrow(res_MPvMC[which(res_MPvMC$log2FoldChange<0 & res_MPvMC$padj<0.05),])), stringsAsFactors=FALSE) #repeat for male
#    DE_female[[i]] <- c(rownames(res_FPvFC[which(res_FPvFC$padj<0.05),]))
#    DE_male[[i]] <- c(rownames(res_MPvMC[which(res_MPvMC$padj<0.05),]))
#}
#save(sex_raref, DE_female, DE_male, file="sex_raref.RData")

Load in the data (since you did this at some point before).

In [ ]:
load("sex_raref.RData")
sex_raref$total <- sex_raref$up + sex_raref$down
sex_raref$sex <- factor(sex_raref$sex)

Get summary stats.

In [ ]:
summary(sex_raref[sex_raref$sex=="F",])
summary(sex_raref[sex_raref$sex=="M",])

In [ ]:
sex_raref_long <- gather(sex_raref, "up", "down", "total", key="direction", value="number")
sex_raref_long$direction <- factor(sex_raref_long$direction, levels=c("up","down","total"))

In [ ]:
ggplot(sex_raref_long, aes(x=number, color=sex)) +
    geom_density(fill="white", alpha=0.5, position="identity") +
    facet_grid(rows= vars(direction), scales="free")

I'll do both a t-test and a Mann-Whitney-Wilcoxon test, because the data are not particularly normally distributed, especially for males.

In [ ]:
wilcox.test(up ~ sex, data=sex_raref)
wilcox.test(down ~ sex, data=sex_raref)
wilcox.test(total ~ sex, data=sex_raref)
t.test(up ~ sex, data=sex_raref)
t.test(down ~ sex, data=sex_raref)
t.test(total ~ sex, data=sex_raref)

Regardless of the test, the difference is extremely significant for all groups (up, down, total).

## Range-specific effects

Now that we have looked for bulk differences as a result of infection status, and how it interacts with sex, we can move on to look at differences among populations with different levels of historical exposure to the parasite. We included the FP samples in our first comparison because it was agnostic to range. However, because it is unresolved whether Loxo is native, invasive, or absent in FP, we are going to remove it from subsequent analyses. Also, we are going to remove the MD samples as well. Three reasons. First, the Loxo used to infect the crabs during the experiment were from MD. It is plausible that these Loxo could be locally adapted to the MD hosts, or vice versa, and that this site-level signal could interfere with the more regional perspective. Second, in the population genomic study of Rhithro, MD clustered with the Northeastern sites where the parasite is absent (NJ, MA, NH). This shared deeper evolutionary history may interfere with questions about more recent adaptations to the presence of the parasite. And third, after removal of FP, both the native and absent ranges have two sites each and thus fewer samples. Exclusion of MD will make sample sizes more even. 

We have to make another dds object. I am going to make it from scratch by removing all FP and MD samples from the coldata and counts_clean.

In [ ]:
colnames(counts_clean)

In [ ]:
counts_clean_noFPMD <- counts_clean[,!grepl("FP_*",colnames(counts_clean))]
counts_clean_noFPMD <- counts_clean_noFPMD[,!grepl("MD_*",colnames(counts_clean_noFPMD))]
coldata_noFPMD <- coldata[colnames(counts_clean_noFPMD),]

Let's just check to make sure they were removed.

In [ ]:
dim(counts_clean_noFPMD)

Just a quick sanity check to make sure the samples are in the same order in the counts and metadata matrices.

In [ ]:
all(rownames(coldata_noFPMD) == colnames(counts_clean_noFPMD))

In [ ]:
rownames(coldata_noFPMD)

Because we have removed the FP and MD samples, I am going to re-normalize the counts and do a second round of expression filtering, using the same process of removing transcripts that have normalized counts less than 10 in over 90% of samples (52 in this case). 

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = counts_clean_noFPMD, colData = coldata_noFPMD, design = ~ condition)
dds <- estimateSizeFactors(dds)
norm_mat <- counts(dds, normalized=TRUE)
dim(norm_mat)
remove <- rowSums(norm_mat < 10 ) > 52
norm_mat <- norm_mat[!remove,]
dim(norm_mat)
counts_clean_noFPMD <- counts_clean_noFPMD[rownames(norm_mat),]

Okay now we're working with 59579 transcripts.

### Comparing infected vs control within ranges, controlling for site

With this pared down dataset, we'll do two different analyses. The first will be just looking at the difference between infected and control *within* each range. This allows us to control for site-specific effects within the ranges. We'll do the same procedure as for the within sex comparison. But first...

There is an added complication when correcting for individual effects that are nested within groups. I am implementing an approach outlined in the ["Group-specific condition effects, individuals nested within groups"](http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#group-specific-condition-effects-individuals-nested-within-groups) section of the DESeq2 vignette. Essentially we just assign a unique identifier to each site by range.

In [ ]:
print(coldata_noFPMD)

We have to add a column assigning unique factors to each population *within* each range. We are going to add a column with the following values:

In [ ]:
coldata_noFPMD$site.n <- factor(c(1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2))
print(coldata_noFPMD)

You can see now that each site has a unique number in relation to other sites in its range. They are repeated across ranges, but it doesn't matter because we are interested in looking at effects of infection within sites. 

Now I'm just going to check the factor levels.

In [ ]:
coldata_noFPMD$sex
coldata_noFPMD$condition
coldata_noFPMD$range
coldata_noFPMD$site

Looks like the condition column retained the old value "F" for the loxo externae and the site column retained the old value "para". Let's also get rid of the sites we removed. Let's relevel.

In [ ]:
coldata_noFPMD$condition <- factor(coldata_noFPMD$condition, c("C","P"))
coldata_noFPMD$site <- factor(coldata_noFPMD$site, c('AP','LA','ML','NH','NJ','SC'))
coldata_noFPMD$condition
coldata_noFPMD$site

Now we can make a model matrix that includes the terms we want to use in our forumlae.

In [ ]:
mm <- model.matrix(~ range + range:site.n + range:condition, coldata_noFPMD)
head(mm)

In [ ]:
colnames(mm)

Let's take a look at the full model matrix.

In [ ]:
print(unname(mm))

In [ ]:
colnames(mm)

Okay now we are going to do the same randomization procedure as we did for the between sex comparison. We are downsampling to 2 for parasitized and 5 for control. NOW I AM GOING TO TRY TO GET TO 3 P AND 6 C PER RANGE. IN NAT, EXTRA C HAS TO COME FROM LA AND EXTRA P FROM AP. IN INV, EXTRA C CAN COME FROM EITHER AND EXTRA P FROM SC. IN ABS, EITHER FOR EITHER.

In [ ]:
#range_raref <- data.frame(rep=integer(0),range=character(0),up=integer(0),down=integer(0), stringsAsFactors = FALSE)
#DE_native <- list() 
#DE_invasive <- list()
#DE_absent <- list()
#for (i in 1:3){
#    coldata_noFPMD_C <- coldata_noFPMD[coldata_noFPMD$condition == "C",] #subset to just control
#    coldata_noFPMD_C <- coldata_noFPMD_C %>% rownames_to_column('sample') %>% group_by(site) %>% sample_n(size=5) %>% column_to_rownames('sample') #group by site and randomly sample 5 (lowest number of C samples in any site)
#    coldata_noFPMD_P <- coldata_noFPMD[coldata_noFPMD$condition == "P",] #subset to just parasitized
#    coldata_noFPMD_P <- coldata_noFPMD_P %>% rownames_to_column('sample') %>% group_by(site) %>% sample_n(size=2) %>% column_to_rownames('sample') #group by site and randomly sample 2 (lowest number of P samples in any site)
#    coldata_noFPMD_raref <- rbind(coldata_noFPMD_C,coldata_noFPMD_P) #combine dfs
#    add_C <- coldata_noFPMD[intersect(rownames(coldata_noFPMD[coldata_noFPMD$condition=="C",]), setdiff(rownames(coldata_noFPMD),rownames(coldata_noFPMD_raref))),] %>% rownames_to_column('sample') %>% group_by(range) %>% sample_n(size=1) %>% column_to_rownames('sample') #randomly select one control sample per range from set of samples not already included 
#    coldata_noFPMD_raref <- rbind(coldata_noFPMD_raref, add_C) #add to df
#    add_P <- coldata_noFPMD[intersect(rownames(coldata_noFPMD[coldata_noFPMD$condition=="P",]), setdiff(rownames(coldata_noFPMD),rownames(coldata_noFPMD_raref))),] %>% rownames_to_column('sample') %>% group_by(range) %>% sample_n(size=1) %>% column_to_rownames('sample') #randomly select one parasitized sample per range from set of samples not already included 
#    coldata_noFPMD_raref <- rbind(coldata_noFPMD_raref, add_P) #add to df
#    counts_clean_noFPMD_raref <- counts_clean_noFPMD[,match(rownames(coldata_noFPMD_raref), colnames(counts_clean_noFPMD))] #subset count data to match included samples
#    mm_raref <-  mm[match(rownames(coldata_noFPMD_raref),rownames(mm)),] #subset model matrix
#    dds_range_raref <- DESeqDataSetFromMatrix(countData = counts_clean_noFPMD_raref, colData = coldata_noFPMD_raref, design = mm_raref)
#    dds_range_raref <- DESeq(dds_range_raref, parallel=TRUE, quiet=TRUE)
#    native.PvC <- results(dds_range_raref, alpha=0.05, parallel=TRUE, name="rangeNative.conditionP")
#    invasive.PvC <- results(dds_range_raref, alpha=0.05, parallel=TRUE, name="rangeInvasive.conditionP")
#    absent.PvC <- results(dds_range_raref, alpha=0.05, parallel=TRUE, name="rangeAbsent.conditionP")
#    range_raref <- rbind(range_raref, list(rep=i,range="native",up=nrow(native.PvC[which(native.PvC$log2FoldChange>0 & native.PvC$padj<0.05),]),down=nrow(native.PvC[which(native.PvC$log2FoldChange<0 & native.PvC$padj<0.05),])), stringsAsFactors=FALSE) #append rep, range, and # up and down sig DE transcripts
#    range_raref <- rbind(range_raref, list(rep=i,range="invasive",up=nrow(invasive.PvC[which(invasive.PvC$log2FoldChange>0 & invasive.PvC$padj<0.05),]),down=nrow(invasive.PvC[which(invasive.PvC$log2FoldChange<0 & invasive.PvC$padj<0.05),])), stringsAsFactors=FALSE) #repeat for invasive
#    range_raref <- rbind(range_raref, list(rep=i,range="absent",up=nrow(absent.PvC[which(absent.PvC$log2FoldChange>0 & absent.PvC$padj<0.05),]),down=nrow(absent.PvC[which(absent.PvC$log2FoldChange<0 & absent.PvC$padj<0.05),])), stringsAsFactors=FALSE) #repeat for absent
#    DE_native[[i]] <- c(rownames(native.PvC[which(native.PvC$padj<0.05),]))
#    DE_invasive[[i]] <- c(rownames(invasive.PvC[which(invasive.PvC$padj<0.05),]))
#    DE_absent[[i]] <- c(rownames(absent.PvC[which(absent.PvC$padj<0.05),]))
#}
#save(range_raref, DE_native, DE_invasive, DE_absent, file='range_raref.RData')

In [ ]:
load("range_raref.RData")
range_raref$total <- range_raref$up + range_raref$down
range_raref$range <- factor(range_raref$range)

In [ ]:
range_raref

In [ ]:
summary(range_raref[range_raref$range=="native",])
summary(range_raref[range_raref$range=="invasive",])
summary(range_raref[range_raref$range=="absent",])

In [ ]:
range_raref_long <- gather(range_raref, "up", "down", "total", key="direction", value="number")
range_raref_long$direction <- factor(range_raref_long$direction, levels=c("up","down","total"))

In [ ]:
ggplot(range_raref_long, aes(x=number, color=range)) +
    geom_density(fill="white", alpha=0.5, position="identity") +
    facet_grid(rows= vars(direction), scales="free") +
    xlim(c(0,3000))

In [ ]:
#wilcox.test(up ~ sex, data=sex_raref)
#wilcox.test(down ~ sex, data=sex_raref)
#wilcox.test(total ~ sex, data=sex_raref)

In the native range, there are 162 upregulated and 249 downregulated, for a total of 411.

In the invasive range, there are 826 upregulated and 403 downregulated, for a total of 1229.

In the absent range, there are 3751 upregulated and 1143 downregulated, for a total of 4894.

Let's save these results to data.frames.

In [ ]:
#native.PvC_df <- data.frame(subset(native.PvC, native.PvC$padj < 0.05))
#native.PvC_df <- native.PvC_df[order(native.PvC_df$padj),]
#invasive.PvC_df <- data.frame(subset(invasive.PvC, invasive.PvC$padj < 0.05))
#invasive.PvC_df <- invasive.PvC_df[order(invasive.PvC_df$padj),]
#absent.PvC_df <- data.frame(subset(absent.PvC, absent.PvC$padj < 0.05))
#absent.PvC_df <- absent.PvC_df[order(absent.PvC_df$padj),]

Let's have a look at some of the top DE transcripts for each range.

In [ ]:
#head(native.PvC_df,20)
#head(invasive.PvC_df,20)
#head(absent.PvC_df,20)

Now I'll make new plotting functions to break it out by range. We'll use modified versions of the `easy_plotCounts()` and `add_title()` functions I wrote above.

In [ ]:
easy_plotCounts_range <- function(ID){
    count_plot <- plotCounts(dds_range, gene=ID, intgroup=c("condition","range", "site"), returnData=TRUE)
    ggplot(count_plot, aes(x=condition, y=count, group=range, color=site)) +
        facet_grid(.~range) +
        theme(strip.text.x = element_text(size = 12), 
              axis.text.x = element_text(size=11), 
              axis.text.y = element_text(size=11)) +
        xlab("") +
        ylab("") +
        geom_point(position=position_jitter(w=0.2,h=0), show.legend = FALSE) + 
        geom_smooth(method = "lm", se=F, aes(group=1), show.legend = FALSE, color = "blue") +
        stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="pointrange", color="red") +
        stat_summary(fun.data=mean_sdl, fun.args = list(mult=1), geom="errorbar", color="red", width=0.2) +
        scale_y_log10()#limits = c(1,1e6)) 
}

add_title_range <- function(ID, title){
     easy_plotCounts_range(ID) + 
        ggtitle(title) + 
        theme(plot.title = element_text(hjust=0.5,size=14,face="bold"),legend.position="none")
}

Just copy paste some transcript IDs to explore.

In [ ]:
annot[annot[,1]=="TRINITY_DN4299_c0_g1_i1",c(1,2,3,11,12,13,14)]
easy_plotCounts_range("TRINITY_DN4299_c0_g1_i1")

Let's count the number of up and down over/under LFC 2, using unshrunk LFCs.

In [ ]:
#nrow(na.omit(native.PvC_df[native.PvC_df$log2FoldChange>2,]))
#nrow(na.omit(native.PvC_df[native.PvC_df$log2FoldChange<(-2),]))
#nrow(na.omit(invasive.PvC_df[invasive.PvC_df$log2FoldChange>2,]))
#nrow(na.omit(invasive.PvC_df[invasive.PvC_df$log2FoldChange<(-2),]))
#nrow(na.omit(absent.PvC_df[absent.PvC_df$log2FoldChange>2,]))
#nrow(na.omit(absent.PvC_df[absent.PvC_df$log2FoldChange<(-2),]))

Now we'll extract the transcript names of those that are up- or downregulated for making Venn diagrams.

In [ ]:
#native_DE_up <- rownames(native.PvC_df[native.PvC_df$log2FoldChange>0,])
#native_DE_down <- rownames(native.PvC_df[native.PvC_df$log2FoldChange<0,])
#invasive_DE_up <- rownames(invasive.PvC_df[invasive.PvC_df$log2FoldChange>0,])
#invasive_DE_down <- rownames(invasive.PvC_df[invasive.PvC_df$log2FoldChange<0,])
#absent_DE_up <- rownames(absent.PvC_df[absent.PvC_df$log2FoldChange>0,])
#absent_DE_down <- rownames(absent.PvC_df[absent.PvC_df$log2FoldChange<0,])
#length(native_DE_up)
#length(native_DE_down)
#length(invasive_DE_up)
#length(invasive_DE_down)
#length(absent_DE_up)
#length(absent_DE_down)

Good, these correspond to the numbers seen earlier in the summaries.

Now to make some Venn diagrams.

In [ ]:
#futile.logger::flog.threshold(futile.logger::ERROR, name = "VennDiagramLogger")
#myCol <- brewer.pal(3, "Set1")
#venn.diagram(
#        x = list(native_DE_up, invasive_DE_up, absent_DE_up),
#        category.names = c("Native" , "Invasive" , "Absent"),
#        filename = '../vis/venn_range_up.png',
#        output = TRUE ,
#        imagetype="png" ,
#        height = 480 , 
#        width = 480 , 
#        resolution = 300,
#        compression = "lzw",
#        lwd = 1,
#        fill = myCol,
#        cex = 0.5,
#        fontfamily = "sans",
#        cat.cex = 0.5,
#        cat.fontface = "bold",
#        cat.default.pos = "outer",
#        cat.pos = c(-27, 27, 180),
#        cat.dist = c(0.055, 0.055, 0.045),
#        cat.fontfamily = "sans",
#        cat.col = myCol,
#        rotation = 1
#)
#venn.diagram(
#        x = list(native_DE_down, invasive_DE_down, absent_DE_down),
#        category.names = c("Native" , "Invasive" , "Absent"),
#        filename = '../vis/venn_range_down.png',
#        output = TRUE ,
#        imagetype="png" ,
#        height = 480 , 
#        width = 480 , 
#        resolution = 300,
#        compression = "lzw",
#        lwd = 1,
#        fill = myCol,
#        cex = 0.5,
#        fontfamily = "sans",
#        cat.cex = 0.5,
#        cat.fontface = "bold",
#        cat.default.pos = "outer",
#        cat.pos = c(-27, 27, 180),
#        cat.dist = c(0.055, 0.055, 0.045),
#        cat.fontfamily = "sans",
#        cat.col = myCol,
#        rotation = 1
#)
#venn.diagram(
#        x = list(c(native_DE_down,native_DE_up), c(invasive_DE_down,invasive_DE_up), c(absent_DE_down,absent_DE_up)),
#        category.names = c("Native" , "Invasive" , "Absent"),
#        filename = '../vis/venn_range_both.png',
#        output = TRUE ,
#        imagetype="png" ,
#        height = 480 , 
#        width = 480 , 
#        resolution = 300,
#        compression = "lzw",
#        lwd = 1,
#        fill = myCol,
#        cex = 0.5,
#        fontfamily = "sans",
#        cat.cex = 0.5,
#        cat.fontface = "bold",
#        cat.default.pos = "outer",
#        cat.pos = c(-27, 27, 180),
#        cat.dist = c(0.055, 0.055, 0.045),
#        cat.fontfamily = "sans",
#        cat.col = myCol,
#        rotation = 1
#)

And an upset plot:

In [ ]:
#require(UpSetR)
#listInput <- list(
#    native_DE_up = native_DE_up,
#    native_DE_down = native_DE_down,
#    invasive_DE_up = invasive_DE_up,
#    invasive_DE_down =  invasive_DE_down,
#    absent_DE_up = absent_DE_up,
#    absent_DE_down = absent_DE_down
#)
#sets <- fromList(listInput)
#upset_plot <- upset(sets, sets = names(listInput), order.by = "freq", nsets=6, mb.ratio=c(0.7,.3))
#png("../vis/upset_range_PvC.png", height=10, width=10, units = "in", res=200)
#upset_plot
#dev.off()
#upset_plot
#detach("package:UpSetR", unload=TRUE)

You can have a look at these in the `vis/` directory. Now on to our last comparison.

### Testing for interactions between range and condition

Here we will investigate differences in the transcriptional response to parasitism *among* ranges. That is, we will search for transcripts that display a significant interaction between `condition` and `range` in the model. Note that here we cannot control for site-specific effects as we did in the last comparison. This is not a flaw in the experimental design; it is simply that there is no way to have one site occur across ranges in order to control for it, i.e. NH cannot simultaneously be in three ranges. This deviates from the mostly biomedical examples from the vignette and other DESeq2 resources, in which they can control for individual/genotype specific effect across different combinations of treatments. 

We'll make a another dds object, this time called `dds_interaction`. We'll use the same counts and sample metadata as in the previous comparison.

In [ ]:
dds_interaction <- DESeqDataSetFromMatrix(countData = counts_clean_noFPMD, colData = coldata_noFPMD, design = ~range + condition + range:condition)

We are going to do a likelihood ratio test for this comparison, comparing the full model specified in the design above, and a reduced model that includes just `range` and `condition`, but no interaction. 

In [ ]:
dds_interaction <- DESeq(dds_interaction, parallel=TRUE, test="LRT", reduced=~range + condition)

In [ ]:
resultsNames(dds_interaction)

Let's save the results.

In [ ]:
res_interaction <- results(dds_interaction, alpha=0.05, parallel=TRUE) #results of full vs. reduced LRT

In [ ]:
summary(res_interaction)

You can see from the summary that there are 4449 transcripts that show a significant interaction.

Let's save these to a data.frame.

In [ ]:
res_interaction_df <- na.omit(data.frame(subset(res_interaction, res_interaction$padj < 0.05)))
res_interaction_df <- res_interaction_df[order(res_interaction_df$padj),] #sort by padj
dim(res_interaction_df)
head(res_interaction_df)

4449 transcripts, around 7.3% of the entire filtered set, display some sort of interaction between range and condition. Seems really high, but remember that we can't control for site-specific effects. Also, population genomic differences between the sites probably also contributes to this, as many contigs in the transcriptome may be supported by crabs only from a certain site or set of sites. 

Let's look at the expression patterns of select transcripts with decapod annotations.

In [ ]:
length(rownames(res_interaction_df[rownames(res_interaction_df) %in% decapod_annot[,1],]))
head(res_interaction_df[rownames(res_interaction_df) %in% decapod_annot[,1],],30)

There are 547 that have annotations to decapod reference sequences. 

Those listed above are the top thirty transcripts by adjusted p-value that have decapod annotations. Copy a transcript ID of interest and paste into `transcript` below to get annotation and plot counts.

In [ ]:
decapod_annot[decapod_annot[,1]=="TRINITY_DN12438_c0_g1_i1",c(1,2,3,11,12,13,14)]
easy_plotCounts_range("TRINITY_DN12438_c0_g1_i1")

Below I am going to export a selection of counts plots of most significant transcripts showing an interaction, using only those with informative decapod annotations (no hypothetical or unknown proteins). 

In [ ]:
p1 <- add_title_range("TRINITY_DN26483_c0_g1_i1", "5-methylcytosine rRNA methyltransferase")
p2 <- add_title_range("TRINITY_DN283_c0_g1_i2", "prosaposin-like")
p3 <- add_title_range("TRINITY_DN83447_c0_g1_i1", "ribosomal protein 24")
p4 <- add_title_range("TRINITY_DN12129_c0_g1_i1", "spectrin alpha chain-like")
p5 <- add_title_range("TRINITY_DN79154_c0_g1_i1", "glucuronokinase 2")
p6 <- add_title_range("TRINITY_DN31392_c0_g1_i1", "Na-dependent nutrient AA transporter 1-like")
p7 <- add_title_range("TRINITY_DN4054_c0_g1_i1", "scaffold attachment factor B1")
p8 <- add_title_range("TRINITY_DN6459_c3_g1_i1", "ATP-dependent RNA helicase DHX8-like")
p9 <- add_title_range("TRINITY_DN9415_c0_g1_i1", "transcription initiation factor TFIID subunit 1-like")
p10 <- add_title_range("TRINITY_DN14078_c0_g1_i1", "RNAPII subunit A C-term phosphatase SSU72-like")
p11 <- add_title_range("TRINITY_DN3237_c0_g1_i1", "flightless 1-like")
p12 <- add_title_range("TRINITY_DN8188_c0_g1_i1", "longitudinals lacking")
p13 <- add_title_range("TRINITY_DN12438_c0_g1_i1", "cyclin-related FAM58A")

library("cowplot")

main <- plot_grid(p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12, nrow=4, ncol=3)
#legend <- get_legend(plot1  + guides(color = guide_legend(nrow = 1)) + theme(legend.position = "bottom"))
#done <- plot_grid(main, legend, nrow=4,ncol=3)

main

save_plot("../vis/DESeq2_top_interaction_genes.pdf", plot=main, nrow=4, ncol=3)

The plot doesn't render well in jupyter. Take a look at the output in `vis/`.

Now we can make contrasts between the interaction terms to see in which ways the responses are different between pairs of ranges.

In [ ]:
res_interaction_AvN <- results(dds_interaction, alpha=0.05, parallel=TRUE, name="rangeAbsent.conditionP", test="Wald")
res_interaction_IvN <- results(dds_interaction, alpha=0.05, parallel=TRUE, name="rangeInvasive.conditionP", test="Wald")
res_interaction_AvI <- results(dds_interaction, alpha=0.05, parallel=TRUE, contrast=list("rangeAbsent.conditionP","rangeInvasive.conditionP"), test="Wald")

In [ ]:
summary(res_interaction_AvN)
summary(res_interaction_IvN)
summary(res_interaction_AvI)

Now we'll place the results in data.frames and show the top transcripts with annotations.

In [ ]:
res_interaction_AvN_df <- na.omit(data.frame(subset(res_interaction_AvN, res_interaction_AvN$padj < 0.05)))
res_interaction_IvN_df <- na.omit(data.frame(subset(res_interaction_IvN, res_interaction_IvN$padj < 0.05)))
res_interaction_AvI_df <- na.omit(data.frame(subset(res_interaction_AvI, res_interaction_AvI$padj < 0.05)))
res_interaction_AvN_df <- res_interaction_AvN_df[order(res_interaction_AvN_df$padj),]
res_interaction_IvN_df <- res_interaction_IvN_df[order(res_interaction_IvN_df$padj),]
res_interaction_AvI_df <- res_interaction_AvI_df[order(res_interaction_AvI_df$padj),]


head(res_interaction_AvN_df[rownames(res_interaction_AvN_df) %in% decapod_annot[,1],],10)
head(res_interaction_IvN_df[rownames(res_interaction_IvN_df) %in% decapod_annot[,1],],10)
head(res_interaction_AvI_df[rownames(res_interaction_AvI_df) %in% decapod_annot[,1],],10)

Let's see if there are any transcripts that show an interaction for all comparison:

In [ ]:
intersect(rownames(res_interaction_AvN_df), intersect(rownames(res_interaction_IvN_df),rownames(res_interaction_AvI_df)))

Appears not. Let's just explore some.

In [ ]:
annot[annot[,1]=="TRINITY_DN283_c0_g1_i2",c(1,2,3,11,12,13,14)]
easy_plotCounts_range("TRINITY_DN283_c0_g1_i2")

This one shows an interesting pattern related to the trematode parasitism. "TRINITY_DN22600_c0_g1_i1"

In [ ]:
annot[annot[,1]=="TRINITY_DN22600_c0_g1_i1",c(1,2,3,11,12,13,14)]
easy_plotCounts_range("TRINITY_DN22600_c0_g1_i1")

This came up as the most significant when contrasting the range:condition interaction between absent and native. You can see that it's mostly driven by the three NH samples. If you were to remove them, you would see that the all ranges would move in the same direction. What I think is most likely is that reads originating from a parasite other than Loxo in the NH crabs (trematode, maybe entoniscid, since it's crustacean?) are mapping to the crab ubiquitin. Thus you see really high read counts. This transcript wasn't removed in the prefiltering stage because overall it didn't exhibit a really negative LFC. Something to be aware of, not sure it's something we can correct for without going through all of the transcripts manually. I guess what one could do is remove all transcripts that have their highest normalized count in one of these three crabs (or top three are in the three, to be more stringent), like we did for the Loxo samples, but I think that's getting a little too far down the rabbit hole.

Make lists of the transcripts IDs to look at directionality.

In [ ]:
absent_up_native_down <- rownames(res_interaction_AvN_df[res_interaction_AvN_df$log2FoldChange > 0,])
absent_down_native_up <- rownames(res_interaction_AvN_df[res_interaction_AvN_df$log2FoldChange < 0,])
invasive_up_native_down <- rownames(res_interaction_IvN_df[res_interaction_IvN_df$log2FoldChange > 0,])
invasive_down_native_up <- rownames(res_interaction_IvN_df[res_interaction_IvN_df$log2FoldChange < 0,])
absent_up_invasive_down <- rownames(res_interaction_AvI_df[res_interaction_AvI_df$log2FoldChange > 0,])
absent_down_invasive_up <- rownames(res_interaction_AvI_df[res_interaction_AvI_df$log2FoldChange < 0,])
length(absent_up_native_down)
length(absent_down_native_up)
length(invasive_up_native_down)
length(invasive_down_native_up)
length(absent_up_invasive_down)
length(absent_down_invasive_up)

In [ ]:
require(UpSetR)
listInput <- list(
    absent_up_native_down = absent_up_native_down,
    absent_down_native_up = absent_down_native_up,
    invasive_up_native_down = invasive_up_native_down,
    invasive_down_native_up = invasive_down_native_up,
    absent_up_invasive_down = absent_up_invasive_down,
    absent_down_invasive_up = absent_down_invasive_up
)
sets <- fromList(listInput)
upset_plot <- upset(sets, sets = names(listInput), order.by = "freq", nsets=6, mb.ratio=c(0.7,.3))
png("../vis/upset_interaction_contrast.png", height=10, width=10, units = "in", res=200)
upset_plot
dev.off()
upset_plot

Below I am just going to make a Venn diagram showing the intersections between AvN, IvN, and AvI. This doesn't specify direction (uses union of range1_down_range2_up, etc.)

In [ ]:
length(absent_down_native_up)
length(absent_up_native_down)
length(c(absent_down_native_up,absent_up_native_down))

In [ ]:
futile.logger::flog.threshold(futile.logger::ERROR, name = "VennDiagramLogger")
venn.diagram(
        x = list(c(absent_up_native_down,absent_down_native_up), 
                 c(invasive_up_native_down,invasive_down_native_up), 
                 c(absent_up_invasive_down,absent_down_invasive_up)),
        category.names = c("A vs. N" , "I vs. N" , "A vs. I"),
        filename = '../vis/venn_interaction_contrast.png',
        output = TRUE ,
        imagetype="png" ,
        height = 480 , 
        width = 480 , 
        resolution = 300,
        compression = "lzw",
        lwd = 1,
        fill = myCol,
        cex = 0.5,
        fontfamily = "sans",
        cat.cex = 0.5,
        cat.fontface = "bold",
        cat.default.pos = "outer",
        cat.pos = c(-27, 27, 180),
        cat.dist = c(0.055, 0.055, 0.045),
        cat.fontfamily = "sans",
        cat.col = myCol,
        rotation = 1
)

Now I am going to make a PCA just with those transcripts that showed a significant interaction in the overall LRT. This should help us get at the similarity/difference in response to infection among ranges. 

In [ ]:
vsd_int <- vst(dds_range, blind=TRUE)

In [ ]:
res_interaction_df_0.01 <- na.omit(data.frame(subset(res_interaction, res_interaction$padj < 0.01)))
vsd_int_sig <- vsd_int[rownames(vsd_int)%in%rownames(res_interaction_df_0.01),]
pcaData <- plotPCA(vsd_int_sig, intgroup=c("condition", "range"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=range, shape=condition)) +
    geom_point(size=3) +
    xlab(paste0("PC1: ",percentVar[1],"% variance")) +
    ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
    coord_fixed()
ggsave("../vis/vsd_interaction_PCA_p0.01.png")
dim(vsd_int_sig)

This is sort of interesting. When looking just at the transcripts that were significant at an alpha threshold of 0.01, the absent uninfected tend to cluster with the invasive infected. When invasive are infected, they tend to fall more in the space of the native or absent uninfected.

Here I am going to try some different plotting to get a summary figure that shows general expression trends among ranges based on the range interaction approach.

First I'm just going to figure out how many transcripts show a significant interaction under various p-value thresholds.

In [ ]:
nrow(res_interaction_df[res_interaction_df$padj<0.05,])
nrow(res_interaction_df[res_interaction_df$padj<0.01,])
nrow(res_interaction_df[res_interaction_df$padj<0.001,])
nrow(res_interaction_df[res_interaction_df$padj<0.0001,])

In [ ]:
interaction_counts <- counts(dds_interaction, normalized = TRUE) #get the normalized counts
interaction_counts <- interaction_counts[rownames(interaction_counts) %in% rownames(res_interaction_df[res_interaction_df$padj<0.01,]),] #select just significant transcripts under given padj
interaction_counts <- t(interaction_counts) #transpose before scaling
interaction_counts <- scale(interaction_counts) #mean-center on zero by stdev
interaction_counts <- as.data.frame(interaction_counts) #convert to df
interaction_counts$pair <- paste(coldata_noFPMD$condition, coldata_noFPMD$range, sep=".") #add column C/P & range ID'ed
interaction_counts <- aggregate(interaction_counts[,1:(ncol(interaction_counts)-1)], list(interaction_counts$pair), mean)
rownames(interaction_counts) <- interaction_counts[,1]
interaction_counts[,1] <- NULL
interaction_counts$condition <- as.factor(c("C","C","C","P","P","P")) #add var for inf status (couldn't figure easy way to split rowname)
interaction_counts$range <- as.factor(c("Absent","Invasive","Native","Absent","Invasive","Native")) #again for range
interaction_counts$range <- factor(interaction_counts$range, levels=c("Native","Invasive","Absent")) #set factor levels so plot facets are ordered as desired
head(interaction_counts)
interaction_counts <- gather(interaction_counts, "gene", "value", 1:(ncol(interaction_counts)-2))
head(interaction_counts, 10)

Okay great we've got it in the right format for plotting. 

In [ ]:
ggplot(interaction_counts, aes(x=condition, y=value, group=range, color=gene)) +
    facet_grid(.~range) +
    geom_path(aes(group=gene),show.legend=FALSE, color="black", alpha=0.05) +
    scale_x_discrete(expand=c(0.1,0.1)) +
    ylim(c(-2,2)) +
    ylab(expression(sigma)) +
    xlab("Infection Status") +
    theme(axis.text=element_text(size=12), 
          axis.title=element_text(size=20),
          strip.text.x = element_text(size = 20))
ggsave("../vis/DE_range_summary.pdf", width = 8, height = 5)

## Export data for `WGCNA` and `GO_MWU`

Before we finish, I need to export a matrix of counts to use downstream in `WGCNA`. The creators [suggest](https://horvath.genetics.ucla.edu/html/CoexpressionNetwork/Rpackages/WGCNA/faq.html) removing transcripts with consistently low counts to avoid spurious correlations, which we have already done, and also recommend performing a variance stabilizing transformation. I'll do the vst on the count tables and then export as a tsv.

I am going to export both one from the P vs. C comparison (w/ 77 samples including FP and MD) and one from the range comparison (w/ 58 samples excluding FP and MD). 

In [ ]:
vsd <- vst(dds_clean, blind=FALSE)
dim(vsd)
write.table(assays(vsd)[[1]], file = "../outputs/WGCNA_PvC.tsv", sep="\t")
vsd <- vst(dds_interaction, blind=FALSE)
dim(vsd)
write.table(assays(vsd)[[1]], file = "../outputs/WGCNA_range.tsv", sep="\t")

Now for the `GO_MWU` data. We'll export both the LFCs and the  pvalues from the base PvC comparison and just the pvalues from the range:condition interaction LRT, because testing between three groups it's unclear to me what the interaction LRT LFCs represent. I'll also export the interaction contrasts pvals and the within range PvC pvals in Fisher format. The p-values need to be -log10, so that bigger numbers represent more significant. They also must be unadjusted. For the genearl PvC and sex PvC, they also need to be directionalized in that for genes that are significantly underexpressed, the final -log10(pvalue) must be negative. For the overall range interaction results, however, I will leave them as all positive numbers and use a one tailed test in GO_MWU. This is because in a three way comparison the LFCs are not informative (as noted above). Keeping the -log10pvals positive let's me test for functions that are enriched among transcripts showing a strong interaction between range and infection status without considering direction of response. In some cases, we'll export for both use in a MWU test and conventional Fisher exact test for enrichment, by substituting 0 for insignificant padj and 1 for significant, at the 0.05 threshold. 

In [ ]:
thresh <- 0.05

#PvC LFCs
GO_MWU_PvC <- na.omit(data.frame(res_clean))
GO_MWU_PvC_LFC <- data.frame(name=rownames(GO_MWU_PvC),LFC=GO_MWU_PvC$log2FoldChange)

#PvC -log10(pval) for MWU
GO_MWU_PvC_pval <- na.omit(data.frame(res_clean)) #data.frame of results without NAs
GO_MWU_PvC_pval$pvalue <- (-1)*log10(GO_MWU_PvC_pval$pvalue) #get -log10 of pval
GO_MWU_PvC_pval$direction <- GO_MWU_PvC_pval$log2FoldChange / abs(GO_MWU_PvC_pval$log2FoldChange) #get LFC direction
GO_MWU_PvC_pval$pvalue <- GO_MWU_PvC_pval$pvalue * GO_MWU_PvC_pval$direction #multiply pval by direction
GO_MWU_PvC_pval <- data.frame(name=rownames(GO_MWU_PvC_pval),pval=GO_MWU_PvC_pval$pvalue) #subset

#PvC Fisher
GO_MWU_PvC_fisher <- na.omit(data.frame(res_clean)) #data.frame of results without NAs
GO_MWU_PvC_fisher <- data.frame(names=rownames(GO_MWU_PvC_fisher), sig=GO_MWU_PvC_fisher$padj) #extract padj!!!!
GO_MWU_PvC_fisher$sig <- GO_MWU_PvC_fisher$sig < thresh #boolean. F = > thres, T = < thresh
GO_MWU_PvC_fisher$sig <- as.numeric(GO_MWU_PvC_fisher$sig) #change to number

#sex separate -log10(pval) for MWU
GO_MWU_FPvFC_pval <- na.omit(data.frame(res_FPvFC))
GO_MWU_FPvFC_pval$pvalue <- (-1)*log10(GO_MWU_FPvFC_pval$pvalue)
GO_MWU_FPvFC_pval$direction <- GO_MWU_FPvFC_pval$log2FoldChange / abs(GO_MWU_FPvFC_pval$log2FoldChange) #get LFC direction
GO_MWU_FPvFC_pval$pvalue <- GO_MWU_FPvFC_pval$pvalue * GO_MWU_FPvFC_pval$direction #multiply pval by direction
GO_MWU_FPvFC_pval <- data.frame(name=rownames(GO_MWU_FPvFC_pval),pval=GO_MWU_FPvFC_pval$pvalue) #subset

GO_MWU_MPvMC_pval <- na.omit(data.frame(res_MPvMC))
GO_MWU_MPvMC_pval$pvalue <- (-1)*log10(GO_MWU_MPvMC_pval$pvalue)
GO_MWU_MPvMC_pval$direction <- GO_MWU_MPvMC_pval$log2FoldChange / abs(GO_MWU_MPvMC_pval$log2FoldChange) #get LFC direction
GO_MWU_MPvMC_pval$pvalue <- GO_MWU_MPvMC_pval$pvalue * GO_MWU_MPvMC_pval$direction #multiply pval by direction
GO_MWU_MPvMC_pval <- data.frame(name=rownames(GO_MWU_MPvMC_pval),pval=GO_MWU_MPvMC_pval$pvalue) #subset

#interaction -log10(pval) for MWU
GO_MWU_interaction_pval <- na.omit(data.frame(res_interaction))
GO_MWU_interaction_pval$pvalue <- (-1)*log10(GO_MWU_interaction_pval$pvalue)
#GO_MWU_interaction_pval$direction <- GO_MWU_interaction_pval$log2FoldChange / abs(GO_MWU_interaction_pval$log2FoldChange) 
#GO_MWU_interaction_pval$pvalue <- GO_MWU_interaction_pval$pvalue * GO_MWU_interaction_pval$direction 
GO_MWU_interaction_pval <- data.frame(name=rownames(GO_MWU_interaction_pval),pval=GO_MWU_interaction_pval$pvalue)

#interaction Fisher
GO_MWU_interaction_fisher <- na.omit(data.frame(res_interaction))
GO_MWU_interaction_fisher <- data.frame(names=rownames(GO_MWU_interaction_fisher), sig=GO_MWU_interaction_fisher$padj)
GO_MWU_interaction_fisher$sig <- GO_MWU_interaction_fisher$sig < thresh
GO_MWU_interaction_fisher$sig <- as.numeric(GO_MWU_interaction_fisher$sig)

#within range PvC -log10(pval) for MWU
GO_MWU_native.PvC_pval <- na.omit(data.frame(native.PvC))
GO_MWU_native.PvC_pval$pvalue <- (-1)*log10(GO_MWU_native.PvC_pval$pvalue)
GO_MWU_native.PvC_pval$direction <- GO_MWU_native.PvC_pval$log2FoldChange / abs(GO_MWU_native.PvC_pval$log2FoldChange) 
GO_MWU_native.PvC_pval$pvalue <- GO_MWU_native.PvC_pval$pvalue * GO_MWU_native.PvC_pval$direction 
GO_MWU_native.PvC_pval <- data.frame(name=rownames(GO_MWU_native.PvC_pval),pval=GO_MWU_native.PvC_pval$pvalue)

GO_MWU_invasive.PvC_pval <- na.omit(data.frame(invasive.PvC))
GO_MWU_invasive.PvC_pval$pvalue <- (-1)*log10(GO_MWU_invasive.PvC_pval$pvalue)
GO_MWU_invasive.PvC_pval$direction <- GO_MWU_invasive.PvC_pval$log2FoldChange / abs(GO_MWU_invasive.PvC_pval$log2FoldChange) 
GO_MWU_invasive.PvC_pval$pvalue <- GO_MWU_invasive.PvC_pval$pvalue * GO_MWU_invasive.PvC_pval$direction 
GO_MWU_invasive.PvC_pval <- data.frame(name=rownames(GO_MWU_invasive.PvC_pval),pval=GO_MWU_invasive.PvC_pval$pvalue)

GO_MWU_absent.PvC_pval <- na.omit(data.frame(absent.PvC))
GO_MWU_absent.PvC_pval$pvalue <- (-1)*log10(GO_MWU_absent.PvC_pval$pvalue)
GO_MWU_absent.PvC_pval$direction <- GO_MWU_absent.PvC_pval$log2FoldChange / abs(GO_MWU_absent.PvC_pval$log2FoldChange) 
GO_MWU_absent.PvC_pval$pvalue <- GO_MWU_absent.PvC_pval$pvalue * GO_MWU_absent.PvC_pval$direction 
GO_MWU_absent.PvC_pval <- data.frame(name=rownames(GO_MWU_absent.PvC_pval),pval=GO_MWU_absent.PvC_pval$pvalue)

#interaction contrasts -log10(pval) for MWU
GO_MWU_interaction_AvN_pval <- na.omit(data.frame(res_interaction_AvN))
GO_MWU_interaction_AvN_pval$pvalue <- (-1)*log10(GO_MWU_interaction_AvN_pval$pvalue)
GO_MWU_interaction_AvN_pval$direction <- GO_MWU_interaction_AvN_pval$log2FoldChange / abs(GO_MWU_interaction_AvN_pval$log2FoldChange) 
GO_MWU_interaction_AvN_pval$pvalue <- GO_MWU_interaction_AvN_pval$pvalue * GO_MWU_interaction_AvN_pval$direction 
GO_MWU_interaction_AvN_pval <- data.frame(name=rownames(GO_MWU_interaction_AvN_pval),pval=GO_MWU_interaction_AvN_pval$pvalue)

GO_MWU_interaction_IvN_pval <- na.omit(data.frame(res_interaction_IvN))
GO_MWU_interaction_IvN_pval$pvalue <- (-1)*log10(GO_MWU_interaction_IvN_pval$pvalue)
GO_MWU_interaction_IvN_pval$direction <- GO_MWU_interaction_IvN_pval$log2FoldChange / abs(GO_MWU_interaction_IvN_pval$log2FoldChange) 
GO_MWU_interaction_IvN_pval$pvalue <- GO_MWU_interaction_IvN_pval$pvalue * GO_MWU_interaction_IvN_pval$direction 
GO_MWU_interaction_IvN_pval <- data.frame(name=rownames(GO_MWU_interaction_IvN_pval),pval=GO_MWU_interaction_IvN_pval$pvalue)

GO_MWU_interaction_AvI_pval <- na.omit(data.frame(res_interaction_AvI))
GO_MWU_interaction_AvI_pval$pvalue <- (-1)*log10(GO_MWU_interaction_AvI_pval$pvalue)
GO_MWU_interaction_AvI_pval$direction <- GO_MWU_interaction_AvI_pval$log2FoldChange / abs(GO_MWU_interaction_AvI_pval$log2FoldChange) 
GO_MWU_interaction_AvI_pval$pvalue <- GO_MWU_interaction_AvI_pval$pvalue * GO_MWU_interaction_AvI_pval$direction 
GO_MWU_interaction_AvI_pval <- data.frame(name=rownames(GO_MWU_interaction_AvI_pval),pval=GO_MWU_interaction_AvI_pval$pvalue)

#interaction contrasts Fisher
GO_MWU_interaction_AvN_fisher <- na.omit(data.frame(res_interaction_AvN))
GO_MWU_interaction_AvN_fisher <- data.frame(names=rownames(GO_MWU_interaction_AvN_fisher), sig=GO_MWU_interaction_AvN_fisher$padj)
GO_MWU_interaction_AvN_fisher$sig <- GO_MWU_interaction_AvN_fisher$sig < thresh
GO_MWU_interaction_AvN_fisher$sig <- as.numeric(GO_MWU_interaction_AvN_fisher$sig)

GO_MWU_interaction_IvN_fisher <- na.omit(data.frame(res_interaction_IvN))
GO_MWU_interaction_IvN_fisher <- data.frame(names=rownames(GO_MWU_interaction_IvN_fisher), sig=GO_MWU_interaction_IvN_fisher$padj)
GO_MWU_interaction_IvN_fisher$sig <- GO_MWU_interaction_IvN_fisher$sig < thresh
GO_MWU_interaction_IvN_fisher$sig <- as.numeric(GO_MWU_interaction_IvN_fisher$sig)

GO_MWU_interaction_AvI_fisher <- na.omit(data.frame(res_interaction_AvI))
GO_MWU_interaction_AvI_fisher <- data.frame(names=rownames(GO_MWU_interaction_AvI_fisher), sig=GO_MWU_interaction_AvI_fisher$padj)
GO_MWU_interaction_AvI_fisher$sig <- GO_MWU_interaction_AvI_fisher$sig < thresh
GO_MWU_interaction_AvI_fisher$sig <- as.numeric(GO_MWU_interaction_AvI_fisher$sig)

Probably should have written a function given how many times I had to do that...

Now let's export them.

In [ ]:
write.csv(GO_MWU_PvC_LFC, file = "../outputs/GO_MWU_PvC_LFC.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_PvC_pval, file = "../outputs/GO_MWU_PvC_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_PvC_fisher, file = "../outputs/GO_MWU_PvC_fisher.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_FPvFC_pval, file = "../outputs/GO_MWU_FPvFC_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_MPvMC_pval, file = "../outputs/GO_MWU_MPvMC_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_interaction_pval, file = "../outputs/GO_MWU_interaction_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_interaction_fisher, file = "../outputs/GO_MWU_interaction_fisher.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_native.PvC_pval, file = "../outputs/GO_MWU_native.PvC_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_invasive.PvC_pval, file = "../outputs/GO_MWU_invasive.PvC_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_absent.PvC_pval, file = "../outputs/GO_MWU_absent.PvC_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_interaction_AvN_pval, file = "../outputs/GO_MWU_interaction_AvN_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_interaction_IvN_pval, file = "../outputs/GO_MWU_interaction_IvN_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_interaction_AvI_pval, file = "../outputs/GO_MWU_interaction_AvI_pval.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_interaction_AvN_fisher, file = "../outputs/GO_MWU_interaction_AvN_fisher.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_interaction_IvN_fisher, file = "../outputs/GO_MWU_interaction_IvN_fisher.csv", quote = FALSE, row.names = FALSE)
write.csv(GO_MWU_interaction_AvI_fisher, file = "../outputs/GO_MWU_interaction_AvI_fisher.csv", quote = FALSE, row.names = FALSE)

Uncomment the next block to save the workspace image for work later, if you desire.

In [ ]:
#save.image()

Print the session info to screen and save a copy to `envs/`.

In [ ]:
sessionInfo()
writeLines(capture.output(sessionInfo()), "../envs/DESeq2_sessionInfo.txt")